Notatki i plan pracy:
1.  Odfiltrowanie "kwantylowe" współrzędnych:
 - rysunek po i przed odfiltrowaniem
2. Podział na typy i wyświetlenie rozkładu przestrzennego:
 - Fire
 - EMS
 - Traffic
3. Dla każdego z typów należy:
 - Oznaczyć elementy odosobnione geograficznie jako niewykorzystywane przez algorytm "k-mean". Proponowanym rozwiązaniem było zastosowanie metody gęstości względnej.  Ze względu na konieczność uwzględnienia bardzo dużego (licznego) otoczenia wykonanie obliczeń zajmuje za dużo czasu. Z powodzeniem zastosowałem algorytm wykrywający zadaną ilość elementów odosobnionych, których średnia odległość do "k" (parametr) najbliższych sąsiadów jest największa.   
 - Na podstawie pozycji GPS na odfiltrowanym (poprzedni punkt) zbiorze danych przeprowadzić proces klasteryzacji z wykorzystaniem algorytmu "k-means". Po uzyskaniu współrzędnych centrów klastrów zakwalifikować wszytkie punkty danych do klastrów oraz narysować klastry wraz z ich centrami.
 - Pogrupować dane po dniach w ramach poszczególnych klastrów.
 - Wykryć elementy odosobnione (mediana + rozstęp ćwiartkowy) jako dni, w których w danym klastrze ilość zdarzeń badanego typu odbiega od normy.
 - Zwizualizować otrzymane wyniki na wspólnej dla wszystkich klastrów osi czasu oraz przedstawić w formie map  w celu przeprowadzenia analizy dni, w których anomalie wystąpiły w kilku sąsiadujących ze sobą klastrach.
 - [Ola] Wyciągnąć zbiory uczący, testory 
 - [Ola] Przeprowadzić uczenie 1-NN
 4. [Ola] Elementy odosobnione dzielnice + mapa porównawcza 
 5. [Ola] Podzielić wyniki na 3 zbiory w relacji do 3 sigma: zbyt czułe, podobnie czułe, nieczułe, wyświetl dane - DLACzEGO tak się dzieje
 
 
 Dokumentacja:
 1. Use case
 2. Uważaj na elementy między klastrami 
 3. analiza zależności występowania elementów odosobnionych w sąsiadujących klastrach (patrz 3).
 
 Pomysły:
    1. Można spróbować przeprowadzić klasyfikacje metodą najbliższego sąsiada przyjmując jako zbiór danych wejściowych wektory zawierające:
       -  Ilość zgłoszeń danego dnia w zależności od typu w badanym klastrze [X_fire,X_ems,X_traffic] , a jako wzorzec potraktować sumę lub alternatywę logiczną wykrycia anomali wcześniej stosowanym algorytmem dla każdego z typów osobno.
       - Ilość zgłoszeń określonego typu, danego dnia z podziałem na klastry [X_kl1, X_kl2, ... X_klN]. Wtedy wzorcem uczącym może być na przykład zbiór wykrytych anomali danego typu bez podziału na regiony lub pewna funkcja logiczna AND/OR agregująca informacje o wykryviu anomali badanego typu pochodzącej z wielu klastrów. Można tutaj rozważyć teź funkcję progową (ilosc_klastrow_z_anomalia > N ? Anomalia: Brak anomali) .
       
  Zwiększyć liczebność zbioru testowego - np. 200 przypadków normy + duplikat razy 5 zbioru anomalii

 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import math
import random
from datetime import datetime

from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
os.mkdir('EMS')
os.mkdir('Traffic')
os.mkdir('Fire')

# Any results you write to the current directory are saved as output.

1. Ładowanie danych:

In [ ]:
df = pd.read_csv("../input/911.csv")
df['timeStamp'] = pd.to_datetime(df['timeStamp'])
print ("Data matrix shape:{}".format(df.shape))

  2. Wyłuskanie z kolumny "title" informacji o typie i podtypie zdarzenia    

In [ ]:
x = df['title'].str.split(":",1,expand=True)
df['type'] = x[0].str.strip(" -")
df['subtype'] = x[1].str.strip(" -")
df

   3. Odfiltrowanie i wyświetlenie elementów nietypowych (mediana + rozstęp ćwiartkowy) ze względu na pozycję GPS kolejno dla długości i szerokości geograficznej - usunięcie błędów geolokalizacji. 

In [ ]:
# Zmodyfikowalem z 2.0 na 2.5 bo przy 2.0 odcinamy fragment terenow hrabstwa
threshold = 2.5

filterForDim = lambda dimName: abs(df[dimName] - desc[dimName]["50%"]) < threshold*(desc[dimName]["75%"]-desc[dimName]["25%"])

desc = df.describe()
dfFiltered = df[filterForDim("lat") & filterForDim("lng")]
    
print("Rejected: {}".format(df.shape[0]-dfFiltered.shape[0]))

ax = df.plot.scatter(x='lat', y='lng', c='r',title="Rozkład przestrzenny zgłoszeń",label = 'Wszystkie',figsize=(8,6))
dfFiltered.plot.scatter(x='lat', y='lng', c='g', label = 'Po odrzuceniu nietypowych', ax = ax)
plt.savefig('unfiltered-outliers.png')
dfFiltered.plot.scatter(x='lat', y='lng', c='r',title="Rozkład przestrzenny po odfiltrowaniu",label = 'Zgłoszenia',figsize=(8,6))
plt.savefig('filtered-outliers.png')
df = dfFiltered

4. Wyświetlenie rozkładu przestrzennego zgłoszeń w zależności od typu.

In [ ]:

uniqueClassList = sorted(list(df['type'].unique()))
colors = ['r','g','b']

ax = None
for index,className in enumerate(uniqueClassList):
    ax = df[df.type == className].plot.scatter(x='lat', y='lng', marker = '.', c=colors[index], label = className,
                                               title="Rozkład przestrzenny zgłoszeń",figsize=(8,6), ax = ax)
    plt.savefig('types-all.png')

for index,className in enumerate(uniqueClassList):
    df[df.type == className].plot.scatter(x='lat', y='lng', marker = '.', c=colors[index], label = className,
                                          title="Rozkład przestrzenny {}".format(className),figsize=(8,6))
    plt.savefig("types-{}.png".format(className))

   5. Liczność zgłoszeń według typu.

In [ ]:
dfType = df.groupby(["type"],as_index=False).count()
dfType.plot.pie(y='lat', labels = dfType['lat'],fontsize=12, figsize=(6,6),title = "Liczność typów.",labeldistance=0.5).legend(dfType["type"])
plt.savefig("types-pie.png")
dfType

In [ ]:
dfDateType = df.groupby(["type", df.timeStamp.map(lambda x: x.date())]).count()
dfDateType.index.names = ["Typ zgłoszeń", "Czas"]

dfDateType.rename(columns={'e':'Ilość zgłoszeń'}, inplace=True)
plt.figure()
dfDateType["Ilość zgłoszeń"].unstack("Typ zgłoszeń").plot(figsize=(16,8), ylim=(0,500))
plt.ylabel('Ilość zgłoszeń')
plt.savefig("types-time.png")

6. Zdefiniowanie parametrów oraz badanego typu.

In [ ]:
activeType = "Traffic"

nNeighbors = 10
nLargest = 1000

kMeanClusters = 10
sampleSize = 40

dfat = df[df.type == activeType].copy()

   7. Przeprowadzenie wykrywania i maskowania (dla algorytmu k-mean) p-elementów odosobnionych,  których średnia odległość do k-sąsiadów jest największa.
   
   Note: Metoda gęstości względnej ze względu na duże skupienie punktów okazuje się nieskuteczna !!! Może ona działać kiedy uwzględni się bardzo duże sąsiedztwo, natomiast uzyskuje się wtedy nieakceptowalny czas obliczeń.

In [ ]:
nn = NearestNeighbors(n_neighbors= nNeighbors)

dfat[['lat','lng']]

nn.fit(dfat[['lat','lng']])
dist,indx = nn.kneighbors() 

dfat['avg_neighbor_dist'] = dist.mean(axis=1)
dfatOutliers=dfat.nlargest(nLargest,'avg_neighbor_dist')

dfat['kmean_hide'] = False
dfat.loc[dfatOutliers.index,'kmean_hide'] = True

ax = dfat[dfat.kmean_hide==False].plot.scatter(x='lat', y='lng', marker = '.', c='g', label = "Elementy zwykłe",title="Wynik wykrywania elementów odosobnionych",figsize=(8,6))
dfat[dfat.kmean_hide].plot.scatter(x='lat', y='lng', marker = '.', c='r', label = "Elementy odosobnione",ax=ax)
plt.savefig("{}-unfiltered-outliers.png".format(activeType))
dfat[dfat.kmean_hide==False].plot.scatter(x='lat', y='lng', marker = '.', c='g', label = "Elementy zwykłe",title="Po usunięciu elementów odosobnionych",figsize=(8,6))
plt.savefig("{}-filtered-outliers.png".format(activeType))

8. Wyznaczenie współrzędnych klastrów z wykorzystaniem algorytmu k-mean.

In [ ]:
kmeans = KMeans(n_clusters=kMeanClusters,random_state=0).fit(dfat.loc[dfat.kmean_hide==False,['lat','lng']].values)

dfat['cluster'] = kmeans.predict(dfat[['lat','lng']])
ax = dfat.plot.scatter(x='lat', y='lng', c = 'cluster',  marker = '.', colormap='viridis', title = 'Podział na klastry', figsize=(10,6))

clusterCenters = pd.DataFrame(kmeans.cluster_centers_)
clusterCenters.plot.scatter(x=0,y=1,c='r', marker = 'x', label = 'centra', ax=ax)

ax.set_xlabel("lat")
ax.set_ylabel("lng")
plt.savefig("{}-clusters.png".format(activeType))

dfat['label'] = dfat['twp'].rank(method='dense', ascending=False)
ax = dfat.plot.scatter(x='lat', y='lng', c = 'label',  marker = '.', colormap='viridis', title = 'Podział na jednostki administracyjne', figsize=(10,6))
ax.set_xlabel("lat")
ax.set_ylabel("lng")

9. Agregacja informacji o ilości zdarzeń w ramach klastra określonego dnia.

In [ ]:
dfatpd = dfat.groupby(['cluster',dfat.timeStamp.map(lambda x: x.date())]).lat.count()

tmp = dfat.copy()
tmp['date'] = dfat.timeStamp.map(lambda x: x.date()) 
tmp['unit'] = 1
pd.pivot_table(tmp, index=["date"], columns=["cluster"], values=["unit"], aggfunc=np.sum)

10. Wyszukiwanie elementów nietypowych wewnątrz klastra.

In [ ]:
stats = []
unusual = []
clusterOutliers = dfatpd.copy()

for i in range(0,kMeanClusters):
    desc = dfatpd[i].describe()
    stats.append(desc)
    unrow = dfatpd[i].map(lambda x: int(abs(x - desc['50%']) > 2*(desc['75%']-desc['25%'])))
    clusterOutliers[i] = unrow
    unusual.append(sum(unrow))    
    
statsdf = pd.DataFrame(stats, index=list(range(0,kMeanClusters)))
statsdf['unusual'] = unusual


allOutliers = dfatpd.copy().reset_index()
allOutliers.cluster = 10
allOutliers = allOutliers.groupby(['cluster', 'timeStamp']).lat.sum()
allStats = allOutliers.describe()
allUn = allOutliers.map(lambda x: int(abs(x - allStats['50%']) > 2*(allStats['75%']-allStats['25%'])))
allUnSum = sum(allUn)
clusterOutliers = clusterOutliers.append(allUn)
allStats['unusual'] = allUnSum
statsdf = statsdf.append(allStats)

clusterOutliers

11. Wyświetlenie wykrytych elementów nietypowych w poszczególnych klastrach.

In [ ]:
clusterOutliersPlot =  pd.DataFrame(clusterOutliers).copy().reset_index()
clusterOutliersPlot['plot_h'] = clusterOutliersPlot['cluster'] + clusterOutliersPlot['lat']

clusterOutliersPlot = clusterOutliersPlot.set_index(['cluster', 'timeStamp'])
clusterOutliersPlot.unstack(level='cluster').plot.line( y='plot_h',figsize = (18, 12))
plt.xlabel("Czas")
plt.ylabel("Elementy odosobnione na klaster")
plt.savefig("{}-outliers-per-cluster.png".format(activeType))

12. Wyświetlenie klastrów zawierających elementy nietypowe.

In [ ]:
clusterOutliersCopy = pd.DataFrame(clusterOutliers).copy().reset_index()
clusterOutliersCopyAll = clusterOutliersCopy[clusterOutliersCopy.cluster == 10]
clusterOutliersCopyAll = clusterOutliersCopy[clusterOutliersCopy.lat == 1]

clusterOutliersCopy = clusterOutliersCopy[clusterOutliersCopy.cluster != 10]
clusterOutliersCopy = clusterOutliersCopy[clusterOutliersCopy.lat == 1]

In [ ]:
clusterOutliersCopy = pd.DataFrame(clusterOutliers).copy().reset_index()
clusterOutliersCopy = clusterOutliersCopy[clusterOutliersCopy.cluster != 10]
clusterOutliersCopy = clusterOutliersCopy[clusterOutliersCopy.lat == 1]
timeStampList = sorted(list(clusterOutliersCopy["timeStamp"].unique()))

tmp = dfat.copy()
tmp['date'] = dfat.timeStamp.map(lambda x: x.date()) 

for timeStamp in timeStampList:
    clusters = list(clusterOutliersCopy[clusterOutliersCopy.timeStamp == timeStamp].cluster)
    pointsFromClusters = tmp[tmp.date == timeStamp]
    pointsFromClusters = pointsFromClusters[pointsFromClusters.cluster.isin(clusters)]
    
    ax = pointsFromClusters.plot.scatter(x='lat', y='lng', c = 'cluster',  marker = '.', colormap='viridis', title = "{} dane z klastrów w których wykryto anomalie".format(timeStamp), figsize=(10,6))
    clusterCenters = pd.DataFrame(kmeans.cluster_centers_)
    clusterCenters.plot.scatter(x=0,y=1,c='r', marker = 'x', label = 'centra klastrów', ax=ax)
    plt.savefig("{}/{}-outliers.png".format(activeType, timeStamp))


In [ ]:
clusterOutliersCopyAll = clusterOutliersCopyAll[clusterOutliersCopyAll.cluster == 10]
clusterOutliersBoth = clusterOutliersCopy.loc[clusterOutliersCopy['timeStamp'].isin(clusterOutliersCopyAll['timeStamp'])]

statsdf[0:9].unusual.sum() - clusterOutliersBoth.lat.sum()


13. Przygotowywanie zbiorów do klasyfikacji

In [ ]:
anomalyLearn = dfatpd[clusterOutliers != 0].sample(sampleSize).repeat(10)
normalLearn = dfatpd[clusterOutliers == 0].sample(sampleSize*10)
anomalyTesting = dfatpd[clusterOutliers != 0].drop(anomalyLearn.index)
normalTesting = dfatpd[clusterOutliers == 0].drop(normalLearn.index)

learnInput = pd.concat([anomalyLearn, normalLearn]).sort_index()
learnOutput = clusterOutliers[learnInput.index].sort_index()

testingInput = pd.concat([anomalyTesting, normalTesting]).sort_index()
testingOutput = clusterOutliers[testingInput.index].sort_index()

14. Klasifikacja (KNN dla K=1)

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(learnInput.values.reshape(-1,1), learnOutput.values) 

predictedOutput = neigh.predict(testingInput.values.reshape(-1,1))

In [ ]:
accuracy = pd.DataFrame()
accuracy['real'] = testingOutput.values
accuracy['predictedKNN'] = predictedOutput
accuracy.index = testingOutput.index
score = neigh.score(testingInput.values.reshape(-1,1), testingOutput)
print(score)

15. Klasyfikacja (Naive Bayessian Classifier)

In [ ]:
gauss = GaussianNB()
gauss.fit(learnInput.values.reshape(-1,1), learnOutput.values) 

predictedOutputNB = gauss.predict(testingInput.values.reshape(-1,1))
accuracy['predictedBayessian'] = predictedOutputNB

gauss.score(testingInput.values.reshape(-1,1), testingOutput)

16. Klasyfikacja (Multinomial Bayessian Classifier)

In [ ]:
multi = MultinomialNB()
multi.fit(learnInput.values.reshape(-1,1), learnOutput.values) 

predictedOutputMultiNB = multi.predict(testingInput.values.reshape(-1,1))
accuracy['predictedMulti'] = predictedOutputMultiNB

multi.score(testingInput.values.reshape(-1,1), testingOutput)

17. Różnica w klasyfikatorach

In [ ]:
accuracyRN = accuracy.copy()
accuracyRN['predictedKNN'] += 1
accuracyRN['predictedBayessian'] += 2
accuracyRN['predictedMulti'] += 3
for cluster in accuracyRN.index.get_level_values('cluster').unique():
    accuracyRN.loc[cluster].plot(y=['real', 'predictedKNN', 'predictedBayessian', 'predictedMulti'], figsize=(36,12))
    plt.xlabel("Czas")
    plt.savefig("{}-outliers-cluster{}.png".format(activeType, cluster))

18. Klasyfikacja per cluster (pomysł 2)

In [ ]:
inOutPd = pd.DataFrame(dfatpd)
inOutPd['anom'] = clusterOutliers
inOutPd = inOutPd.reset_index()

inputList = inOutPd.groupby('timeStamp')['lat'].apply(list)
for ind, row in inputList.iteritems():
    while len(row) < 10:
        row.append(0)
outputList = inOutPd.groupby('timeStamp')['anom'].any()

anomalyLearnM = inputList[outputList].sample(sampleSize).repeat(10)
normalLearnM = inputList[~outputList].sample(sampleSize*10)
anomalyTestingM = inputList[outputList].drop(anomalyLearnM.index)
normalTestingM = inputList[~outputList].drop(normalLearnM.index)

learnInputM = pd.concat([anomalyLearnM, normalLearnM]).sort_index()
learnOutputM = outputList[learnInputM.index].sort_index()

testingInputM = pd.concat([anomalyTestingM, normalTestingM]).sort_index()
testingOutputM = outputList[testingInputM.index].sort_index()

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(list(learnInputM), list(learnOutputM))

predictedOutputM = neigh.predict(list(testingInputM))
neigh.score(list(testingInputM), list(testingOutputM))

In [ ]:
multiPd = pd.DataFrame()
multiPd['real'] = testingOutputM + 0
multiPd['pred'] = predictedOutputM + 1

multiPd.plot(y=['real', 'pred'], figsize=(36,12))
plt.savefig("{}-outliers-manycoord.png".format(activeType))

    19. Anomalie a jednostki administracyjne

In [ ]:
dfatadm = dfat.groupby(['twp',dfat.timeStamp.map(lambda x: x.date())]).lat.count()


In [ ]:

stats = []
unusual = []
townOutliers = dfatadm.copy()
towns = list(dfatadm.index.levels[0])

for town in towns:
    desc = dfatadm[town].describe()
    stats.append(desc)
    unrow = dfatadm[town].map(lambda x: int(abs(x - desc['50%']) > 2*(desc['75%']-desc['25%'])))
    townOutliers[town] = unrow
    unusual.append(sum(unrow))    
    
statstwp = pd.DataFrame(stats, index=towns)
statstwp['unusual'] = unusual

statstwp

20. Estymatory jądrowe

In [ ]:
def GaussianKernel(x):
    return 1/math.sqrt(2*math.pi)*math.exp(-math.pow(x, 2)/2)

In [ ]:
def KernelEstimator(x, randomx, m, h):
    return 1/(m*h) * sum(map(lambda xi: GaussianKernel((x-xi)/h), randomx))

In [ ]:



preEstimatorM = pd.DataFrame()
preEstimatorM['real'] = testingOutputM
preEstimatorM['pred'] = predictedOutputM

dateList = preEstimatorM.index.values
xArr = list(map(lambda x: int(datetime(*(x.timetuple())[0:6]).timestamp()), dateList))
xArr = list(map(lambda x: x-xArr[0], xArr))
m = 9
random_sample = np.random.choice(xArr, m)


In [ ]:

h = 0.00001
yArr = list(map(lambda x: KernelEstimator(x, random_sample, m, h), xArr))


preEstimatorM['estimated'] = pd.Series(yArr, dateList)
preEstimatorM = preEstimatorM.fillna(0)

#preEstimatorM[preEstimatorM.pred].estimated.plot(figsize=(36,12))

plt.figure(figsize=(36,12))
plt.plot(xArr, yArr)

In [ ]:

h = 100
yArr = list(map(lambda x: KernelEstimator(x, random_sample, m, h), xArr))


preEstimatorM['estimated'] = pd.Series(yArr, dateList)
preEstimatorM = preEstimatorM.fillna(0)

preEstimatorM[preEstimatorM.pred].estimated.plot(figsize=(36,12))

In [ ]:
xArr1 = np.where(preEstimatorM.pred.values)[0]
m = 40
random_sample = np.random.choice(xArr1, m)
xArr2 = np.where(preEstimatorM.real.values)[0]
random_sample2 = np.random.choice(xArr2, m)

In [ ]:

h = 1
yArr1 = list(map(lambda x: KernelEstimator(x, random_sample, m, h), xArr1))
yArr2 = list(map(lambda x: KernelEstimator(x, random_sample2, m, h), xArr2))
plt.figure(figsize=(36,12))
plt.plot(xArr1, yArr1, xArr2, yArr2)

In [ ]:
h = 8

# yArr1 = list(map(lambda x: KernelEstimator(x, range(640), m, h), xArr1))
# yArr2 = list(map(lambda x: KernelEstimator(x, random_sample2, m, h), xArr2))

y = np.zeros(640)
x = np.zeros(640)
y2 = np.zeros(640)
for i in range(640):
    x[i] = i
    estVal = 0
    estVal2 = 0
    for outlierX in xArr1:
        estVal += GaussianKernel((outlierX - x[i])/h)
    for outlierX in xArr2:
        estVal2 += GaussianKernel((outlierX - x[i])/h)
    estVal /= (len(xArr1)*h)
    estVal2 /= (len(xArr2)*h)
    y2[i] = estVal2
    y[i] = estVal
    
plt.figure(figsize=(36,12))
plt.plot(x,y,x,y2)